In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
sys.path.append("../../project_functions/")
from matplotlib import pyplot as plt
import ad_functions as adfns
import sipa_rep_functions as sf
import sipa09_functions as s9fns
import pytesseract
import cv2
import numpy as np
import pandas as pd
import os
import shutil
from icecream import ic

# psm 6 = Assume a single uniform block of text.
config_tesseract = '--tessdata-dir ./ttesseract_langs --psm 7 digits'

In [4]:
from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import Table
from reportlab.pdfgen import canvas

In [5]:
file_path = "./cropped_images/9/"
file_list = os.listdir(file_path)

file_list, len(file_list)

(['S09_F01_Cropped.png',
  'S09_F02_Cropped.png',
  'S09_F03_Cropped.png',
  'S09_F04_Cropped.png',
  'S09_F05_Cropped.png',
  'S09_F06_Cropped.png',
  'S09_F07_Cropped.png',
  'S09_F08_Cropped.png',
  'S09_F09_Cropped.png',
  'S09_F10_Cropped.png',
  'S09_F11a_Cropped.png',
  'S09_F11b_Cropped.png',
  'S09_F12_Cropped.png',
  'S09_F13_Cropped.png',
  'S09_F14_Cropped.png',
  'S09_F15_Cropped.png',
  'S09_F16_Cropped.png',
  'S09_F17_Cropped.png',
  'S09_F18_Cropped.png',
  'S09_F19_Cropped.png'],
 20)

In [6]:
report_filename = "sipa_09_crnn_600k_report.pdf"

adfns.delete_file(report_filename)


# create a canvas object with A4 landscape page size
pdf = canvas.Canvas(report_filename, pagesize=landscape(A4))
pdf.setTitle('SIPA 9 CRNN Report')

height, width = A4

# set the fill color of the canvas to light grey
pdf.setFillColorRGB(0.8, 0.8, 0.8)

# draw a rectangle to fill the entire page
pdf.rect(0, 0, width, height, fill=True, stroke=False)



heightList = [
    height * 15 / 100, # header
    height * 80 / 100, # body
    height * 5  / 100, # footer
]

horizontalPadding   = 10
verticalPadding     = 10

body_horizontal_padding = 30

length      = len(file_list)
chunckSize  = 2 # number of elements per page



image_matrix = np.empty((0, 7, 300, 300, 3), dtype=np.uint8)
for startIdx in range(0, length, chunckSize):

    endIdx = startIdx + chunckSize
    _data = file_list[startIdx:endIdx]
    image_matrix, digits_matrix = s9fns.get_sipa09_crnn_images(_data, file_path)

    # print("image_matrix.shape", image_matrix.shape)
    # print("digits_matrix.shape", digits_matrix.shape)

    mainTable = Table([
        [sf.genHeaderTable(width, heightList[0], "SIPA 09 Image Processing Report")],
        [sf.s09_crnn_genBodyTable(width - body_horizontal_padding, heightList[1], image_matrix, digits_matrix, _data, chunckSize)],
        [sf.genFooterTable(width, heightList[2])],
    ],

    colWidths=width,
    rowHeights=heightList
    )
    mainTable.setStyle([
        # ('GRID', (0, 0), (-1, -1), 5, 'red'),
        ('VALIGN', (0, 1), (0, 1), 'TOP'),  # vertical align
        ('BOTTOMPADDING', (0, 0), (-1, -1), 0),
        ('LEFTPADDING', (0, 0), (-1, -1), 0),
        # ('LEFTPADDING', (1, 1), (-1, 1), 80), # body horizontal padding
        ('BOTTOMPADDING', (0, 1), (-1, 1), 40),
        ('TOPPADDING', (0, 1), (-1, 1), 15),
        # ('ALIGN', (0, 1), (-1,0), 'CENTER'),
        # ('RIGHTPADDING', (1, 0), (1, -1), 40),
        # ('LEFTPADDING', (1, 0), (1, 0), body_horizontal_padding / 2),
    ])


    mainTable.wrapOn(pdf, 0, 0)
    mainTable.drawOn(pdf, 0, 0)
    pdf.showPage()

pdf.save()


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'sipa_09_crnn_600k_report.pdf'